In [1]:
import numpy as np
import sympy as sp
from control import matlab
from control import canonical_form

In [2]:
A = np.array([[-3, 4],[-1, 5]])
B = np.array([[1],[0]])
C = np.array([[0,2]])
D = np.array([[0]])
Pss = matlab.ss(A, B, C, D)

# 可制御正準系への変換
- ※可観測行列には下の方に$a_n$が集まるタイプと上の方に集まる二種類がる
    - 教科書の表現は下の方に集まるタイプ
    - python-controlでは上の方に集まるタイプで実装されてる
        - 参考：https://en.wikibooks.org/wiki/Control_Systems/Standard_Forms#Controllable_Canonical_Form
    

In [3]:
Pr, T = canonical_form(Pss, form = 'reachable') # オプション'reachable'をつけると可制御正準系になる

In [4]:
print(Pr)

A = [[ 2. 11.]
     [ 1.  0.]]

B = [[1.]
     [0.]]

C = [[-0. -2.]]

D = [[0.]]



In [5]:
P1 = matlab.ss2tf(Pss) #伝達関数に変換
P2 = matlab.ss2tf(Pr)
print(P1, P2) #伝達関数的には等しい事を確認


-8.882e-16 s - 2
----------------
 s^2 - 2 s - 11
 
4.441e-16 s - 2
---------------
s^2 - 2 s - 11



- ↑分子の$s$の項はどっちも糞小さい係数になってるから無視（数値計算上こうなる？）

# 対角正準系への変換
- 可制御正準系と同じような感じでできる
- ゼロの部分はゼロにはならずすごく小さい数になる
- **対角化不可能な系に対してもエラー吐かずむりやり対角化しちゃうっぽいので注意**

In [6]:
A = np.array([[-3, 8], [-3, 7]])
Pss = matlab.ss(A, B, C, D)
Pm, T = canonical_form(Pss, form = 'modal') # オプション'modal'をつけると対角正準系になる

In [7]:
print(Pm)

A = [[3.00000000e+00 7.63648021e-16]
     [5.79790797e-16 1.00000000e+00]]

B = [[ 2.5       ]
     [-3.35410197]]

C = [[-1.2        -0.89442719]]

D = [[0.]]



In [8]:
A = np.array([[1, -9], [1, -5]]) #対角化不可の場合も無理やり計算してしまう　
Pss = matlab.ss(A, B, C, D)
Pm, T = canonical_form(Pss, form = 'modal')
print(Pm)

A = [[-2.00000000e+00  4.65288609e-08]
     [-2.48143998e-08 -2.00000000e+00]]

B = [[1.05409255e+00]
     [6.79637885e+07]]

C = [[ 6.32455532e-01 -9.80914516e-09]]

D = [[0.]]



# その他有用な行列操作
- sympyを使えば、対角化できない場合に「Matrix is not diagonalizable」とエラーが帰って来るのでこっちで確かめると安全

In [9]:
A = np.array([[-3, 8], [-3, 7]])
sp.init_printing()
A = sp.eye(2)*A #sympyの行列定義がめんどいからnumpy.arrayにsimpyの単位行列をかけてる

In [10]:
A.eigenvects() #固有ベクトル　結果は(固有値, 重根かどうか{１：重根でない、２：二重根、みたいな},固有ベクトル)のように帰ってくるっぽい

⎡⎛      ⎡⎡2⎤⎤⎞  ⎛      ⎡⎡4/3⎤⎤⎞⎤
⎢⎜1, 1, ⎢⎢ ⎥⎥⎟, ⎜3, 1, ⎢⎢   ⎥⎥⎟⎥
⎣⎝      ⎣⎣1⎦⎦⎠  ⎝      ⎣⎣ 1 ⎦⎦⎠⎦

In [11]:
A.diagonalize() #対角化（右）※左は対角化するための変換行列T T^{-1}ATで対角化される

⎛⎡2  4⎤  ⎡1  0⎤⎞
⎜⎢    ⎥, ⎢    ⎥⎟
⎝⎣1  3⎦  ⎣0  3⎦⎠

In [12]:
# numpyで検算
A = np.array([[-3, 8], [-3, 7]])
T = np.array([[2,4],[1,3]])
np.dot(np.dot(np.linalg.inv(T), A),T)

array([[1., 0.],
       [0., 3.]])

In [13]:
A = np.array([[1, -9], [1, -5]])  #対角化不可能な場合（エラー吐く）
A = sp.eye(2)*A
A.diagonalize()

MatrixError: Matrix is not diagonalizable

In [14]:
A.jordan_form() #対角化不可能なときはジョルダン行列になおす

⎛⎡3  1⎤  ⎡-2  1 ⎤⎞
⎜⎢    ⎥, ⎢      ⎥⎟
⎝⎣1  0⎦  ⎣0   -2⎦⎠